In [1]:
import sys
## sys.path.append("/mnt/c/Users/iago.louzado/Downloads/backup/Scripts/golden/src")

from golden_old import *
import pandas as pd
import numpy as np
import google

In [2]:
# Iniciando o client para acessar o BQ
client_bq = config_client('prod')

# Verificando se o client está conectado
check_client_connection(client_bq)

/mnt/c/Users/anderson.paiva/Documents/Github/axpm_projetos_dados/meu_ambiente/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


True

In [3]:
# Declarando a query que deseja utilizar
query_bq = """
with base_analitica as (
select current_date() as dt_extracao_base_analitica,
       cm.id_atendimento,
       cm.numero_atendimento,
       cm.data_origem_date,
       cm.nome_pet,
       cm.nome_tutor,
       date_trunc(cm.data_origem_date, month) as ano_mes,
       cm.nome_procedimento,
       cm.vl_bruto,
       cm.valor_copart,
       cm.local_pgmnto_copart,
       if(trim(c.situacao_pagamento) = 'Pago', 'Pago', 'Pendente') as status_pagamento,
       if(date_trunc(cm.data_origem_date, month) >= date_sub(date_trunc(current_date(), month), INTERVAL 3 month), 1, 0) as fl_3_meses,
       dt_quitacao as dt_pagamento,
       status_novo
from `petlove-dataeng-prod-01.op_health.custos_medicos` cm
left join `petlove-dataeng-prod-01.op_health.dts_contabil` c
on cm.id_contabil = c.id_contabil
where cm.sistema_origem_custo = 'ERP Nofaro'
),
-- Queremos uma base que pegue o que estava em aberto no dia 08/01
-- Para isso faremos um union de duas bases:
  -- Está com o status de pendente e teve emissão nos meses 10/11/12
  -- Está com o status de pago mas a data de pagamento é posterior ao dia 07/01
base_status_pendente_copart as (
select data_origem_date,
       ano_mes,
       nome_pet,
       nome_tutor,
       numero_atendimento,
       id_atendimento,
       local_pgmnto_copart,
       nome_procedimento,
       status_pagamento,
       valor_copart,
       0 as vl_bruto
from base_analitica
where 1=1 
and local_pgmnto_copart = 'cartao cadastrado'
and status_pagamento = 'Pendente'
and date_trunc(data_origem_date, month) between '2024-12-01' and '2024-12-01'
and status_novo = 'Aprovado'
),
base_pago_atual_copart as (
  select data_origem_date,
         ano_mes,
         nome_pet,
         nome_tutor,
         numero_atendimento,
         id_atendimento,
         local_pgmnto_copart,
         nome_procedimento,
         status_pagamento,
         valor_copart,
         0 as vl_bruto
  from base_analitica
  where 1=1 
  and local_pgmnto_copart = 'cartao cadastrado'
  and status_pagamento = 'Pago'
  and dt_pagamento > '2025-01-07'
  and status_novo = 'Aprovado'
  and date_trunc(data_origem_date, month) between '2024-12-01' and '2024-12-01'
),
-- base_pago_atual_copart_dez as (
--   select data_origem_date,
--          ano_mes,
--          nome_pet,
--          nome_tutor,
--          numero_atendimento,
--          id_atendimento,
--          local_pgmnto_copart,
--          nome_procedimento,
--          status_pagamento,
--          valor_copart,
--          0 as vl_bruto
--   from base_analitica
--   where 1=1 
--   and local_pgmnto_copart = 'cartao cadastrado'
--   and status_pagamento = 'Pago'
--   and dt_pagamento > '2025-02-07'
--   and status_novo = 'Aprovado'
--   and date_trunc(data_origem_date, month) = '2024-12-01'
-- ),
base_status_pendente_repasse as (
select data_origem_date,
       ano_mes,
       nome_pet,
       nome_tutor,
       numero_atendimento,
       id_atendimento,
       local_pgmnto_copart,
       nome_procedimento,
       status_pagamento,
       0 as valor_copart,
       vl_bruto
from base_analitica
where 1=1 
and status_pagamento = 'Pendente'
and date_trunc(data_origem_date, month) between '2024-12-01' and '2024-12-01'
and status_novo = 'Aprovado'
),
base_pago_atual_repasse as (
  select data_origem_date,
         ano_mes,
         nome_pet,
         nome_tutor,
         numero_atendimento,
         id_atendimento,
         local_pgmnto_copart,
         nome_procedimento,
         status_pagamento,
         0 as valor_copart,
         vl_bruto
  from base_analitica
  where 1=1 
  and status_pagamento = 'Pago'
  and dt_pagamento > '2025-01-07'
  and status_novo = 'Aprovado'
  and date_trunc(data_origem_date, month) between '2024-12-01' and '2024-12-01'
),
-- base_pago_atual_repasse_dez as (
--   select data_origem_date,
--          ano_mes,
--          nome_pet,
--          nome_tutor,
--          numero_atendimento,
--          id_atendimento,
--          local_pgmnto_copart,
--          nome_procedimento,
--          status_pagamento,
--          0 as valor_copart,
--          vl_bruto
--   from base_analitica
--   where 1=1 
--   and status_pagamento = 'Pago'
--   and dt_pagamento > '2025-02-07'
--   and status_novo = 'Aprovado'
--   and date_trunc(data_origem_date, month) = '2024-12-01'
-- ),
base_uniao as (
  select *
  from base_status_pendente_copart
  union all
  select *
  from base_pago_atual_copart
  -- union all
  -- select *
  -- from base_pago_atual_copart_dez
  union all
  select *
  from base_status_pendente_repasse
  union all
  select *
  from base_pago_atual_repasse
  -- union all
  -- select *
  -- from base_pago_atual_repasse_dez
),
final as (
select current_date() as dt_extracao,
       data_origem_date as dt_lancamento,
       ano_mes as ano_mes_lancamento,
       nome_pet,
       nome_tutor,
       numero_atendimento,
       id_atendimento,
       local_pgmnto_copart as pagamento_da_copart,
       nome_procedimento,
       status_pagamento,
       valor_copart,
       vl_bruto as valor_repasse
from base_uniao
)
select * from final
"""

In [4]:
# Declarando a query que deseja utilizar
query_ben = """
select distinct
  b.id_tutor,
  b.nome_tutor,
  b.nome_pet,
  b.num_chip,
  b.email
  -- count(distinct b.id_pet_bu) as num_pets
from `petlove-dataeng-prod-01.op_health.beneficiarios` b
where 1=1
  and b.fl_ativo = 1
"""

In [5]:
# Rodando a query e salvando em um dataset
df2 = run_query(query_bq, client_bq)

Quer matar o Cesar????
Esta consulta processará 2.69 GB.


Deseja executar a consulta? (Y/N) Y


Iniciando query
Iniciando a consulta: 2025-04-06 19:17:49
Consulta concluída: 2025-04-06 19:19:41
Duração da consulta: 0:01:51.986850


In [15]:
# Visualizando o dataframe
df2

,dt_extracao,dt_lancamento,ano_mes_lancamento,nome_pet,nome_tutor,numero_atendimento,id_atendimento,pagamento_da_copart,nome_procedimento,status_pagamento,valor_copart,valor_repasse
0,2025-04-06,2024-12-23,2024-12-01,Piaf,Tammy,5694384,11870377,clinica,Sessão de fisioterapia,Pago,0.0,120.00
1,2025-04-06,2024-12-20,2024-12-01,Wini,Mariana Melzer,5658266,11795638,cartao cadastrado,Parasitológico de Fezes Seriado (3 amostras),Pago,0.0,59.83
2,2025-04-06,2024-12-19,2024-12-01,Bidu,Mariana,5646668,11772810,clinica,Retorno com Especialista - Até 30 Dias,Pago,0.0,62.54
3,2025-04-06,2024-12-03,2024-12-01,Nala,Joyce,5420697,11309485,clinica,Retorno com Especialista - Até 30 Dias,Pago,0.0,50.00
4,2025-04-06,2024-12-23,2024-12-01,Vilma Pequeno,Sonia,5702045,11885712,clinica,Teste de Fluoresceína + Jones bilateral,Pago,0.0,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1118103,2025-04-06,2024-12-13,2024-12-01,Dandy,Roxolana,5561810,11599259,clinica,Vacina da Raiva,Pendente,0.0,16.12
1118104,2025-04-06,2024-12-08,2024-12-01,Eduardo,Amanda,5488305,11447529,clinica,Vacina da Raiva,Pendente,0.0,21.32
1118105,2025-04-06,2024-12-19,2024-12-01,Sátis,Arieni,5650488,11780551,clinica,Vacina da Raiva,Pendente,0.0,12.00
1118106,2025-04-06,2024-12-28,2024-12-01,Oliver,LIVIA DE CASTRO PEREIRA AMARO,5757182,11996775,clinica,Vacina da Raiva,Pendente,0.0,12.00


In [6]:
# Rodando a query e salvando em um dataset
df_ben = run_query(query_ben, client_bq)

Esta consulta processará 138.46 MB.


Deseja executar a consulta? (Y/N) Y


Iniciando query
Iniciando a consulta: 2025-04-06 19:19:49
Consulta concluída: 2025-04-06 19:20:28
Duração da consulta: 0:00:39.044551


In [14]:
# Visualizando o dataframe
df_ben

,id_tutor,nome_tutor,nome_pet,num_chip,email
0,507681.0,Giulliano Fattori,Joe,981020004268518,giulliano_fattori@hotmail.com
1,691386.0,Silvana Ribeiro Costa,Zeus,963003100348717,silvana.rcosta@gmail.com
2,869237.0,Rafaela Ramos Felisardo,Caramela,981020004237333,rafaelarfelisardo@gmail.com
3,560560.0,Danielly Camily Pereira Cabanha,Lola,981020004410729,daniellycamily@hotmail.com
4,808653.0,Laís Silva,Dom,963003100431824,laiscorreasilva32@gmail.com
...,...,...,...,...,...
851932,176427.0,Eliana Vieira,Thor,963003001081908,elianavieira1992@gmail.com
851933,865328.0,Alisson Lucas Medeiros,Thor,963003100496377,alissonlm95@gmail.com
851934,375763.0,Nadja Cibelle Machado Rocha,Thor,981020002947302,nadjacibelle@gmail.com
851935,437113.0,Fernando Silva Skyrda,Thor,981020004218183,fskyrda@hotmail.com


In [ ]:
# Extraindo os dados para um xlsx
# save_to_csv(df2, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'ben_20240406')

In [7]:
df1 = pd.read_csv('/mnt/c/Users/anderson.paiva/Downloads/base_analitica_deloitte_20240403.csv')
df1 = df1.loc[df1['ano_mes_lancamento'] == '2024-12-01']

In [19]:
# Visualizando o dataframe
df1

,dt_extracao,dt_lancamento,ano_mes_lancamento,nome_pet,nome_tutor,numero_atendimento,id_atendimento,pagamento_da_copart,nome_procedimento,status_pagamento,valor_copart,valor_repasse
0,2025-04-04,2024-12-23,2024-12-01,Dudu,Virgilio Russo Spena Visentini,5698426,11878612,clinica,Sessão de fisioterapia,Pago,0.0,150.00
1,2025-04-04,2024-12-27,2024-12-01,Bianca,Veronica,5735140,11952993,clinica,Retorno especialista - até 30 dias,Pago,0.0,52.50
2,2025-04-04,2024-12-16,2024-12-01,Capile,Nathalia,5596713,11672140,clinica,Teste de Fluoresceína + Jones bilateral,Pago,0.0,10.00
3,2025-04-04,2024-12-02,2024-12-01,Jimmy,Simone,5358916,11181718,cartao cadastrado,Histopatológico,Pago,0.0,80.00
5,2025-04-04,2024-12-17,2024-12-01,Baby,Selma,5617261,11713617,cartao cadastrado,Análise citológica Aspirativa (BAAF/CAAF),Pago,0.0,88.83
...,...,...,...,...,...,...,...,...,...,...,...,...
1240489,2025-04-04,2024-12-20,2024-12-01,NALA,LUCAS,5670532,11820608,clinica,Vacina da Raiva,Pendente,0.0,18.85
1240491,2025-04-04,2024-12-09,2024-12-01,Teddy,Flavia,5501847,11475758,clinica,Vacina da Raiva,Pendente,0.0,10.87
1240493,2025-04-04,2024-12-27,2024-12-01,Tomé,MARCELO,5736667,11956286,clinica,Vacina da Raiva,Pendente,0.0,12.00
1240497,2025-04-04,2024-12-02,2024-12-01,Loola,Eliane,5396997,11260848,clinica,Vacina da Raiva,Pendente,0.0,12.00


In [20]:
colunas_ignoradas = ['dt_extracao', 'nome_procedimento']

# Garante que as colunas existem e tira só as que você quer comparar
colunas_para_comparar = [col for col in df1.columns if col not in colunas_ignoradas]

df1_comp = df1[colunas_para_comparar].copy()
df2_comp = df2[colunas_para_comparar].copy()

In [21]:
# Comparando como strings
df1_comp = df1_comp.astype(str)
df2_comp = df2_comp.astype(str)

In [22]:
# Merge com indicador
df_diff = df1_comp.merge(df2_comp, how='outer', indicator=True)

In [23]:
# Pegando apenas linhas diferentes
diferencas = df_diff[df_diff['_merge'] != 'both']

In [24]:
diferencas

,dt_lancamento,ano_mes_lancamento,nome_pet,nome_tutor,numero_atendimento,id_atendimento,pagamento_da_copart,status_pagamento,valor_copart,valor_repasse,_merge
146,2024-12-01,2024-12-01,Alexa Ramalho de Melo,Andressa Maria Ramalho dos Santos Andressa Ram...,5392766,11251326,cartao cadastrado,Pago,0.0,22.0,right_only
147,2024-12-01,2024-12-01,Alexa Ramalho de Melo,Andressa Maria Ramalho dos Santos Andressa Ram...,5392766,11251326,cartao cadastrado,Pago,12.0,0.0,right_only
148,2024-12-01,2024-12-01,Alexa Ramalho de Melo,Andressa Maria Ramalho dos Santos Andressa Ram...,5392766,11251326,cartao cadastrado,Pendente,0.0,22.0,left_only
149,2024-12-01,2024-12-01,Alexa Ramalho de Melo,Andressa Maria Ramalho dos Santos Andressa Ram...,5392766,11251326,cartao cadastrado,Pendente,12.0,0.0,left_only
150,2024-12-01,2024-12-01,Alexa Ramalho de Melo,Andressa Maria Ramalho dos Santos Andressa Ram...,5392767,11251327,clinica,Pago,0.0,5.0,right_only
...,...,...,...,...,...,...,...,...,...,...,...
1117636,2024-12-31,2024-12-01,Toddy Nelson,Ariana Szczepanski,5776370,12036416,cartao cadastrado,Pago,12.0,0.0,right_only
1117637,2024-12-31,2024-12-01,Toddy Nelson,Ariana Szczepanski,5776370,12036417,cartao cadastrado,Pago,0.0,8.0,right_only
1117638,2024-12-31,2024-12-01,Toddy Nelson,Ariana Szczepanski,5776370,12036417,cartao cadastrado,Pago,12.0,0.0,right_only
1117639,2024-12-31,2024-12-01,Toddy Nelson,Ariana Szczepanski,5776370,12036418,cartao cadastrado,Pago,0.0,8.0,right_only


In [ ]:
# Extraindo os dados para um xlsx
# save_to_csv(df_filtro, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'base_exemplo')

In [29]:
diferencas_df1 = diferencas.merge(df1_comp, how='left', on=colunas_para_comparar)
diferencas_df1_vf = diferencas_df1.merge(df_ben, how='left', on=['nome_tutor', 'nome_pet'])

diferencas_df2 = diferencas.merge(df2_comp, how='left', on=colunas_para_comparar)
diferencas_df2_vf = diferencas_df2.merge(df_ben, how='left', on=['nome_tutor', 'nome_pet'])

In [32]:
# Extraindo os dados para um xlsx
save_to_csv(diferencas_df1_vf, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'df1_vf')
save_to_csv(diferencas_df2_vf, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'df2_vf')

'Arquivo df2_vf.csv salvo com sucesso em /mnt/c/Users/anderson.paiva/Downloads/df2_vf.csv!!'